- Q!: 如何使用用户标签来指导业务（如何提升业务）: 标签化的第一目的就是通过用户画像与我们的业务相关联，提升业务的转化率， 减少目标客户的流失。 通常对标签的管理涉及到了主要三个层面对标签何其产生数据的管理和配置： 数据层，算法层，业务层。 在三个层面上分别都具有使用用户标签的方式提升业务的各个方面。a. 数据层是对标签数据的事实状态，因此更加清晰更加明确地存储架构会让后续的算法建模，业务逻辑处理具有十分重要的作用（ 用户行为的记录是否清晰， 用户消费的具体记录，清晰明细的用户标签对用户归类产生很大影响。 b. 随之明确地用户标签记录在算法层对用户数据的建模就更好地预测，归类， 推荐用户更加适合用户的内容商品偏好。 c. 通过建模，使用对预测 归类 的建模结果从而推出更加合适合理吸引用户的消费方式， 商品推荐类型，从而使用户使用本项目的时间更长，消费更多，从而提升项目业务的变现转化率，客户留存率等等。

- Q2: 对于数据打标签的方法 ：基于用户打标签。 基于内容打标签。   实际算法可用 隔各个聚类方法: kmeans  em gmm 等等

- Q3 : 准确率和精确率有何不同 : 
    P精确率是针对我们预测结果的预测准确率 是预测正确的数量和实际正确数量的比
    A准确率是测试数据集，分类器正确分类的样本数与总样本数之比。
    
- Q4: 用户标签的原理: (根据课程) 标签设计像可以用用户兴趣、偏好、人口统计学特征，主要目的是提升营销精准度、推荐匹配度，终极目的是提升产品服务，起到提升企业利润。用户画像适合于各个产品周期：从新用户的引流到潜在用户的挖掘、从老用户的培养到流失用户的回流等目标去查找利用各方面的标签，比如在q1 中说的用户行为等等。    餐厅的话，比如就餐环境，菜品， 服务态度，餐馆位置等等。


- Q5: 这10这种方式都是单独单例的方法。下一步可以尝试浅层融合： 简单粗暴，将最后的prediction 结果 数值化后 取均值。   深层次 可以尝试将多个弱的分类器转化成强分类器 （ adaboost 泛化集成） 